## Implementation of Metropolis Hastings to approximate probability queries from a Bayesian Network.

In [17]:
import numpy as np
from calculations_helper import break_up_polytree, handle_dag_metropolis_hastings, join_distributions

def estimate_metropolis_hastings(p: float, iterations: int, network: dict, queries: list[int], evidence: dict[int,bool]) -> np.array:
    """Generate an estimate for the probability distribution for a given set of query variables and evidence values

    Args:
        p (float): determines probability of generating the next state via either Gibbs Sampling or Likelihood Weighting
        iterations (int): number of samples to take before we go with the estimate
        network (dict): underlying bayesian network
        queries (list[int]): list of query variables
        evidence (dict[int,bool]): list of evidence variables with their respective values

    Returns:
        np.array: estimated probability distribution for the different combinations the query variables can take on
    """
    dag_map, query_collections, evidence_collections = break_up_polytree(network, queries, evidence)
    
    if len(dag_map) == 1:
        # only one directed acyclic graph
        return queries, handle_dag_metropolis_hastings(iterations, p, network, queries, evidence)
    else:
        # each directed acyclic graph will output a probability distribution - we must join them all and keep track of the variables present
        reordered_queries = []
        prob_distributions = []
        for i, dag in dag_map.items():
            these_queries = query_collections[i]
            for v in these_queries:
                reordered_queries.append(v)
            this_evidence = {v:evidence[v] for v in evidence_collections[i]}
            prob_distributions.append(handle_dag_metropolis_hastings(iterations, p, dag, these_queries, this_evidence))
        return reordered_queries, join_distributions(prob_distributions)

In [ ]:
import json
import time

query_list = [[1, 3], [4, 5, 7], [2,6,10,12]]
evidence_list = [{2:False, 5:True}, {2:True, 3:False, 6:False}, {3:False, 4:False, 5:True, 15:True}]

times = []
results = []

for queries,evidence in zip(query_list,evidence_list):
    with open('small_polytree.json') as f:
        bayesian_network = json.load(f)
        start_time = time.perf_counter()
        results.append(estimate_metropolis_hastings(10000, bayesian_network, queries, evidence))
        end_time = time.perf_counter()
        times.append(end_time-start_time)
for res, t in zip(results, times):
    print(f"Variables={res[0]}\nProbabilities={res[1]}\nRuntime={t}\n\n")

In [ ]:
import json
import time

query_list = [[1, 3], [4, 5, 7], [2,6,10,12]]
evidence_list = [{2:False, 5:True}, {2:True, 3:False, 6:False}, {3:False, 4:False, 5:True, 15:True}]

times = []
results = []

for queries,evidence in zip(query_list,evidence_list):
    with open('big_polytree.json') as f:
        bayesian_network = json.load(f)
        start_time = time.perf_counter()
        results.append(estimate_metropolis_hastings(10000, bayesian_network, queries, evidence))
        end_time = time.perf_counter()
        times.append(end_time-start_time)
for res, t in zip(results, times):
    print(f"Variables={res[0]}\nProbabilities={res[1]}\nRuntime={t}\n\n")

([1, 3], array([0.1973437 , 0.32385414, 0.23585324, 0.24294891]))
([1, 3], array([0.19916366, 0.31220531, 0.25466806, 0.23396297]))
([1, 3], array([0.20207879, 0.30299126, 0.2428159 , 0.25211404]))


In [ ]:
import json
import time

query_list = [[1, 3], [4, 5, 7], [2,6,10,12]]
evidence_list = [{2:False, 5:True}, {2:True, 3:False, 6:False}, {3:False, 4:False, 5:True, 15:True}]

times = []
results = []

for queries,evidence in zip(query_list,evidence_list):
    with open('giant_polytree.json') as f:
        bayesian_network = json.load(f)
        start_time = time.perf_counter()
        results.append(estimate_metropolis_hastings(10000, bayesian_network, queries, evidence))
        end_time = time.perf_counter()
        times.append(end_time-start_time)
for res, t in zip(results, times):
    print(f"Variables={res[0]}\nProbabilities={res[1]}\nRuntime={t}\n\n")

([1, 3], array([0.27625956, 0.24145218, 0.24485337, 0.2374349 ]))
([1, 3], array([0.27474401, 0.23484631, 0.24821169, 0.24219799]))
([1, 3], array([0.25674414, 0.24618258, 0.24983827, 0.24723501]))
